In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import pickle

from sklearn.model_selection import train_test_split

from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, Conv1D, GlobalMaxPool1D, LSTM
from tensorflow.keras.models import load_model
import tensorflow as tf

import tensorflow as tf
from tensorflow.compat.v1 import ConfigProto
from tensorflow.compat.v1 import InteractiveSession

In [2]:
config = ConfigProto()
config.gpu_options.allow_growth = True
session = InteractiveSession(config=config)

In [3]:
df = pd.read_csv('../data/raw_recording_1.csv')

df = df[:int(len(df) * 100/100)]

In [4]:
df['x_diff'] = df['x_pos'].diff()
df['y_diff'] = df['y_pos'].diff()

In [5]:
df.dropna(inplace=True)

In [6]:
# for i in range(1,500):

#     df[f'channel_1_{i}'] = df['channel_1'].shift(i)
#     df[f'channel_2_{i}'] = df['channel_2'].shift(i)
#     df[f'channel_3_{i}'] = df['channel_3'].shift(i)
#     df[f'channel_4_{i}'] = df['channel_4'].shift(i)
#     df[f'x_pos_-{i}'] = df['x_pos'].shift(i)
#     df[f'y_pos_-{i}'] = df['y_pos'].shift(i)
#     df[f'x_diff_-{i}'] = df['x_diff'].shift(i)
#     df[f'y_diff_-{i}'] = df['y_diff'].shift(i)

In [7]:
df.dropna(inplace=True)

In [8]:
df.reset_index(drop=True).head()

,x_pos,y_pos,channel_1,channel_2,channel_3,channel_4,x_diff,y_diff
0,2268.0,661.0,22,32,0,83,-4.0,2.0
1,2265.0,664.0,22,31,12,83,-3.0,3.0
2,2265.0,664.0,22,31,16,111,0.0,0.0
3,2265.0,664.0,21,31,16,111,0.0,0.0
4,2265.0,664.0,21,31,16,11,0.0,0.0


In [9]:
df = df[df['x_diff'] != 0]
df = df[df['y_diff'] != 0]


# df = df[df['x_diff'] >= 0]
# df = df[df['y_diff'] >= 0]


# df = df[df['x_diff'] <= 0]
# df = df[df['y_diff'] <= 0]

In [10]:
df.head()

,x_pos,y_pos,channel_1,channel_2,channel_3,channel_4,x_diff,y_diff
1,2268.0,661.0,22,32,0,83,-4.0,2.0
2,2265.0,664.0,22,31,12,83,-3.0,3.0
6,2260.0,667.0,21,31,16,287,-5.0,3.0
8,2257.0,670.0,22,31,15,287,-3.0,3.0
10,2254.0,673.0,21,31,15,287,-3.0,3.0


In [11]:
# def categorize(x, category):
#     if category == '0':
#         if x == 0:
#             return 1
#         else:
#             return 0
#     if category == '1':
#         if (x > 0 and x <=1):
#             return 1
#         else:
#             return 0
#     if category == '-1':
#         if (x < 0 and x >=-1):
#             return 1
#         else:
#             return 0
#     if category == '2':
#         if x > 1:
#             return 1
#         else:
#             return 0
#     if category == '-2':
#         if x < -1:
#             return 1
#         else:
#             return 0





# df['x_diff_cat_0'] = [categorize(val, '0') for val in df['x_diff']]
# df['y_diff_cat_0'] = [categorize(val, '0') for val in df['y_diff']]
# df['x_diff_cat_1'] = [categorize(val, '1') for val in df['x_diff']]
# df['y_diff_cat_1'] = [categorize(val, '1') for val in df['y_diff']]
# df['x_diff_cat_-1'] = [categorize(val, '-1') for val in df['x_diff']]
# df['y_diff_cat_-1'] = [categorize(val, '-1') for val in df['y_diff']]
# df['x_diff_cat_2'] = [categorize(val, '2') for val in df['x_diff']]
# df['y_diff_cat_2'] = [categorize(val, '2') for val in df['y_diff']]
# df['x_diff_cat_-2'] = [categorize(val, '-2') for val in df['x_diff']]
# df['y_diff_cat_-2'] = [categorize(val, '-2') for val in df['y_diff']]


In [12]:
X = np.array(df.drop(columns=['x_pos', 'y_pos', 'x_diff', 'y_diff']))
# y = np.array(df[['x_diff_cat_0', 'y_diff_cat_0',
#                 'x_diff_cat_1', 'y_diff_cat_1',
#                 'x_diff_cat_-1', 'y_diff_cat_-1',
#                 'x_diff_cat_2', 'y_diff_cat_2',
#                 'x_diff_cat_-2', 'y_diff_cat_-2']])
y = np.array(df[['x_diff', 'y_diff']])

In [13]:
X

array([[ 22,  32,   0,  83],
       [ 22,  31,  12,  83],
       [ 21,  31,  16, 287],
       ...,
       [ 13,  28,   6, 352],
       [ 13,  28,   6, 351],
       [ 13,  28,   6, 350]], dtype=int64)

In [14]:
# max(df['x_diff']), min(df['y_diff']), max(df['y_diff']), min(df['y_diff'])

In [15]:
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state = 42)

In [16]:
model = Sequential()

model.add(LSTM(2048, input_shape = (X.shape[1],1)))
model.add(Dense(128, activation='relu'))
model.add(Dense(32, activation = 'relu'))
model.add(Dense(8, activation = 'relu'))
model.add(Dense(2, activation=tf.keras.activations.linear))

model.compile(optimizer = 'adam', loss = tf.keras.losses.mean_squared_error, metrics = ['acc'])


Instructions for updating:
If using Keras pass *_constraint arguments to layers.


In [17]:
model.fit(X_train.reshape((X_train.shape[0],X_train.shape[1],1,)), y_train,
          validation_data = (X_test.reshape((X_test.shape[0],X_test.shape[1],1,)), y_test), epochs = 5)

Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where
Train on 18724 samples, validate on 6242 samples
Epoch 1/5
18724/18724 [==============================] - 530s 28ms/sample - loss: 47.8233 - acc: 0.4713 - val_loss: 20.1934 - val_acc: 0.4696
Epoch 2/5
  416/18724 [..............................] - ETA: 8:11 - loss: 807.7878 - acc: 0.4736

KeyboardInterrupt: 

In [ ]:
plt.figure(figsize=(100,100))

plt.plot(model.predict(X.reshape((X.shape[0],X.shape[1],1,)))[:,0][:10000])
plt.plot(y[:,0][:10000], alpha = .5)

In [ ]:
plt.figure(figsize=(100,100))

plt.plot(model.predict(X.reshape((X.shape[0],X.shape[1],1,)))[:,1][:10000])
plt.plot(y[:,1][:10000], alpha = .5)

In [ ]:
# model.save('../pickled_models/pos_nn_2')